### CIFAR10 数据增强和迁移学习训练流程

- 设计并实现一个包含数据增强和迁移学习的训练流程。
- 使用 TensorBoard 监控训练过程。
- 调整模型和训练参数，直到达到目标准确率。实现 CIFAR10 准确率 > 75% 的模型。

In [ ]:
# 安装 TensorBoard
%pip install tensorboard

# 导入必要的库
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.tensorboard import SummaryWriter

# 数据增强和预处理
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# 加载 CIFAR10 数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# 加载预训练的 ResNet 模型
model = models.resnet18(pretrained=True)

# 修改最后的全连接层以适应 CIFAR10 数据集（10 类）
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)

# 定义损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 初始化 TensorBoard
writer = SummaryWriter('runs/cifar10_experiment')

# 训练模型并记录损失值和准确率
for epoch in range(20):  # 训练 20 个 epoch
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if i % 100 == 99:  # 每 100 个 mini-batch 打印一次损失
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            writer.add_scalar('training loss', running_loss / 100, epoch * len(trainloader) + i)
            running_loss = 0.0
    accuracy = 100 * correct / total
    writer.add_scalar('training accuracy', accuracy, epoch)

print('Finished Training')
writer.close()

In [ ]:
# 评估模型性能并记录准确率
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the 10000 test images: {accuracy} %')
writer.add_scalar('test accuracy', accuracy)
writer.close()

### 启动 TensorBoard

在命令行中运行以下命令以启动 TensorBoard：

```sh
tensorboard --logdir=runs
```

然后在浏览器中打开 http://localhost:6006 查看训练过程的可视化结果。